In [28]:
import sqlite3
import csv
# import pandas as pd
import numpy as np
import pandas as pd
import json

In [29]:
class DataBaseInfo():
    def __init__(self, tableName : str, schema : dict, cursor = None):
        self.tableName = tableName
        self.schema = schema
        self.createTable(cursor)


    def createTable(self, cursor):
        createTQuery = f"CREATE TABLE IF NOT EXISTS {self.tableName} ("
        for field, type in self.schema.items():
            createTQuery += f" {field} {type},"
        createTQuery = createTQuery[:-1]
        createTQuery += ");"
        if (cursor):
            cursor.execute(createTQuery)
        else:
            print(createTQuery)

    
    def csvToDB(self, csv_file, cursor):
        with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)  # Skip header row if needed
            if (cursor):
                for i, row in enumerate(csv_reader):
                    row = ([str(i+1)]+row[1:])
                    insertQuery = f"INSERT INTO my_table VALUES {tuple(row)}"
                    cursor.execute(insertQuery)
            else:
                for i, row in enumerate(csv_reader):
                    row = ([str(i+1)]+row[1:])
                    insertQuery = f"INSERT INTO my_table VALUES {tuple(row)}"
                    print(insertQuery)

    def csvToDF(self, csv_file):
        self.df = pd.read_csv(csv_file)



In [30]:
schema_CCR = json.load("schema_ccr.json")

AttributeError: 'str' object has no attribute 'read'

In [31]:
schema_CCR = {
    "SNO"               : "INTEGER",
    "CO_ID"             : "TEXT",
    "CO_Date"           : "DATE", 
    "BLOCK"             : "TEXT",
    "BLOCK_CODE"        : "TEXT",
    "EQUIPMENT"         : "TEXT",
    "EQUIPMENT_CODE"    : "TEXT",
    "INSTRUMENT"        : "TEXT",
    "INSTRUMENT_CODE"   : "TEXT",
    "CO_STATUS"         : "TEXT",
    "CALIB_CAT"         : "DATE",
    "CALIB_SUMMARY"     : "TEXT",
    "COMP_DATE"         : "DATE",
    "COMP_BY"           : "TEXT",
    "EXP_DATE_OF_CLOSURE" : "DATE",
    "APPROVAL_STATUS"   : "TEXT",
    "APPROVED_BY"       : "TEXT",
    "DAYS_TO_COMP"      : "INTEGER",
    "MONTH"             : "INTEGER",
    "DATE"              : "INTEGER",
    "MGR_RESP"          : "TEXT",
    "SR_EXE_RESP"       : "TEXT"
}

In [3]:
schema = {'SNO': 'INTEGER', 'EQUIPMENT':'TEXT', 'AREA' : 'TEXT', 'SCHEDULED_DATE' : 'DATE', 'RESCHEDULED' : 'DATE', 'COMPLETED_ON' : 'DATE',}

In [4]:
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

In [5]:
dbObj = DataBaseInfo('my_table', schema = schema, cursor=cursor)

In [6]:
dbObj.csvToDB('./trial-data/test-data.csv', cursor = cursor)

In [7]:
dbObj.csvToDF('./trial-data/test-data.csv')

In [8]:
dbObj.df.head()

,Index,Equipment,Area,Scheduled-Date,Rescheduled,Completed-on
0,NaN,TSD-M101,WPB,10/11/2023,NaN,10/11/2023
1,NaN,TSD-M102,WPB,10/29/2023,NaN,10/29/2023
2,NaN,TSD-M103,WPB,10/25/2023,NaN,10/25/2023
3,NaN,TSD-M104,WPB,10/4/2023,NaN,10/4/2023
4,NaN,TSD-M105,WPB,10/8/2023,NaN,10/8/2023


In [9]:
len(dbObj.df)

98

In [10]:
dff = dbObj.df.copy()

In [17]:
dff['Scheduled-Date'] = pd.to_datetime(dff['Scheduled-Date'])
dff['Rescheduled'] = pd.to_datetime(dff['Rescheduled'])
dff['Completed-on'] = pd.to_datetime(dff['Completed-on'])

In [11]:
def applyFilter(dbObj, filters : dict):
    # Query from SQL
    # filter dict {'field' : [lowerLimit, upperLimit]} both limits inclusive
    df = dbObj.df 
    for field, limits in filters.item():
        df = df[(df[field]>= limits[0]) & (df[field]<= limits[1])]
    return df


In [24]:
def computeScheduleCompliance(df, meta : dict):
    # meta['frequency'] = [daily, weekly, monthly, yearly]
    # meta['metric'] = ['percentage']
    # meta['KPI'] = ['calibration', 'PM'] (optional)
    # add logic here
    # returns json
    result_json = {}
    df['Scheduled-Date'] = pd.to_datetime(df['Scheduled-Date'])
    df['Completed-on'] = pd.to_datetime(df['Completed-on'])

    if meta['frequency'] == 'daily':
        for month in df['Scheduled-Date'].dt.month.unique():
            month = int(month)
            if (month > 0):
                df_temp = df[df['Scheduled-Date'].dt.month == month]
                total = len(df_temp)
                comp = len(df_temp[df_temp['Scheduled-Date'] == df_temp['Completed-on']])
                result_json[month] = (comp/total) * 100    
    elif meta['frequency'] == 'weekly':
        pass
    elif meta['frequency'] == 'monthly':
        for year in df['Scheduled-Date'].dt.year.unique():
            year = int(month)
            df_temp = df[df['Scheduled-Date'].dt.year == year]
            for month in df_temp['Scheduled-Date'].dt.month.unique():
                month = int(month)
                if (month > 0):
                    df_temp = df_temp[df_temp['Scheduled-Date'].dt.month == month]
                    total = len(df_temp)
                    comp = len(df_temp[df_temp['Scheduled-Date'] == df_temp['Completed-on']])
                    result_json[month] = (comp/total) * 100
    elif meta['frequency'] == 'yearly':
        pass
    return json.dumps(result_json)


In [25]:
dff

,Index,Equipment,Area,Scheduled-Date,Rescheduled,Completed-on
0,NaN,TSD-M101,WPB,2023-10-11,NaN,2023-10-11
1,NaN,TSD-M102,WPB,2023-10-29,NaN,2023-10-29
2,NaN,TSD-M103,WPB,2023-10-25,NaN,2023-10-25
3,NaN,TSD-M104,WPB,2023-10-04,NaN,2023-10-04
4,NaN,TSD-M105,WPB,2023-10-08,NaN,2023-10-08
...,...,...,...,...,...,...
93,NaN,MIX-M233,SPB,2023-10-12,NaN,2023-10-12
94,NaN,MIX-M234,SPB,2023-10-06,NaN,2023-10-06
95,NaN,MIX-M235,SPB,2023-10-07,NaN,2023-10-07
96,NaN,MIX-M236,SPB,2023-10-04,NaN,2023-10-04


In [26]:
meta = {}
meta['frequency'] = 'monthly'

In [27]:
computeScheduleCompliance(dff, meta=meta)

'{"10": 77.31958762886599, "9": 0.0}'

In [ ]:
def computeDailyCompliance():
    # add logic here
    # returns json
    return None